In this notebook we compare ICA and PCA in terms of source reconstruction capability.

We first mix the national anthems of Austria, Germany and Russia and try to reconstruct them using each method.

In [26]:
import scipy.io.wavfile
import numpy as np
from numpy.linalg import matrix_rank,inv
from sklearn.decomposition import PCA,FastICA
import matplotlib.pyplot as plt

First we read the wavfiles for the 3 anthems, check their shapes and store their values

In [27]:
german_anthem=scipy.io.wavfile.read("anthem1.wav")

In [28]:
signal_ger=german_anthem[1]
german_anthem[1].shape

(3065789, 2)

In [29]:
austrian_anthem=scipy.io.wavfile.read("anthem2.wav")

In [30]:
signal_aus=austrian_anthem[1]
austrian_anthem[1].shape

(3704826, 2)

In [31]:
russian_anthem=scipy.io.wavfile.read("anthem3.wav")

In [32]:
signal_rus=russian_anthem[1]
russian_anthem[1].shape

(3087000, 2)

In [33]:
print("Sampling rate GER=",german_anthem[0])
print("Sampling rate AUS=",austrian_anthem[0])
print("Sampling rate RUS=",russian_anthem[0])

Sampling rate GER= 44100
Sampling rate AUS= 44100
Sampling rate RUS= 44100


The sampling rate is equal for all signals

Now we combine the left channels for all signals in a matrix stacking the sources on the columns. 
The length of the signals is truncated according to the shortest one which is the german anthem

In [34]:
ger_left_channels=signal_ger[:,0].reshape(len(signal_ger[:,0]),1)

aus_left_channels=signal_aus[:,0].reshape(len(signal_aus[:,0]),1)
aus_left_channels=aus_left_channels[:len(signal_ger[:,0]),]

rus_left_channels=signal_rus[:,0].reshape(len(signal_rus[:,0]),1)
rus_left_channels=rus_left_channels[:len(signal_ger[:,0]),]

Y=np.c_[ger_left_channels,aus_left_channels,rus_left_channels]

In [35]:
print(Y.shape)

(3065789, 3)


Next, we apply an arbitrary (full rank) mixing matrix to mix the 3 signals together

In [36]:
#mixing signals

#A=np.random.randint(5,size=(3,3)) #mixing matrix
A=np.array([[1, 1, 1], [0.5, 2, 1.0], [1.5, 1.0, 2.0]])

X=np.dot(Y,A.T) # mixed signals

In [37]:
X.shape

(3065789, 3)

In [38]:
matrix_rank(A) # checking that A has full rank

3

Now, we scale the mixed signals between -1 and 1, crop them to ten seconds and finally write them to .wav files and listen to them 

In [39]:
sample_rate=german_anthem[0]

s1=X[:,0]
s2=X[:,1]
s3=X[:,2]

mixed1=2*(s1-s1.min())/(s1.max()-s1.min())-1
mixed1=mixed1[:sample_rate*10]

mixed2=2*(s2-s2.min())/(s2.max()-s2.min())-1
mixed2=mixed2[:sample_rate*10]

mixed3=2*(s3-s3.min())/(s3.max()-s3.min())-1
mixed3=mixed3[:sample_rate*10]

In [40]:
#writing to wavfiles
scipy.io.wavfile.write("mixed_sources1.wav",sample_rate,mixed1)
scipy.io.wavfile.write("mixed_sources2.wav",sample_rate,mixed2)
scipy.io.wavfile.write("mixed_sources3.wav",sample_rate,mixed3)

**Reconstruction using ICA**

We now try to reconstruct our signals to obtain our original sources using ICA, by extracting 3 components

In [41]:
#Reconstruction using ICA

ica=FastICA(n_components=3)
Sources_ica=ica.fit_transform(X)

In [42]:
Sources_ica.shape

(3065789, 3)

We now scale the reconstructed signals between -1 and 1, crop them to 10 seconds and write them to wav files to be able to listen to them

In [43]:
#Scaling and cropping
src1_ica=2*(Sources_ica[:,0]-Sources_ica[:,0].min())/(Sources_ica[:,0].max()-Sources_ica[:,0].min())-1
src1_ica=src1_ica[:sample_rate*10]

src2_ica=2*(Sources_ica[:,1]-Sources_ica[:,1].min())/(Sources_ica[:,1].max()-Sources_ica[:,1].min())-1
src2_ica=src2_ica[:sample_rate*10]

src3_ica=2*(Sources_ica[:,2]-Sources_ica[:,2].min())/(Sources_ica[:,2].max()-Sources_ica[:,2].min())-1
src3_ica=src3_ica[:sample_rate*10]

In [44]:
#writing to wavfiles
scipy.io.wavfile.write("reconstructed_ica1.wav",sample_rate,src1_ica)
scipy.io.wavfile.write("reconstructed_ica2.wav",sample_rate,src2_ica)
scipy.io.wavfile.write("reconstructed_ica3.wav",sample_rate,src3_ica)

It is clear from listening to the files that our origninal sources are almost prefectly reconstructed using ICA and we now have 3 files with almost non-overlapping anthems

**Reconstruction using PCA**

We now try to do reconstruction using PCA by extracting 3 components and checking the results

In [45]:
#Reconstruction using PCA

pca = PCA(n_components=3)
Sources_pca=pca.fit_transform(X)

We now scale the reconstructed signals between -1 and 1, crop them to 10 seconds and write them to wav files to be able to listen to them

In [46]:
#Scaling and cropping
src1_pca=2*(Sources_pca[:,0]-Sources_pca[:,0].min())/(Sources_pca[:,0].max()-Sources_pca[:,0].min())-1
src1_pca=src1_pca[:sample_rate*10]

src2_pca=2*(Sources_pca[:,1]-Sources_pca[:,1].min())/(Sources_pca[:,1].max()-Sources_pca[:,1].min())-1
src2_pca=src2_pca[:sample_rate*10]

src3_pca=2*(Sources_pca[:,2]-Sources_pca[:,2].min())/(Sources_pca[:,2].max()-Sources_pca[:,2].min())-1
src3_pca=src3_pca[:sample_rate*10]

In [47]:
#writing to wavfiles
scipy.io.wavfile.write("reconstructed_pca1.wav",sample_rate,src1_pca)
scipy.io.wavfile.write("reconstructed_pca2.wav",sample_rate,src2_pca)
scipy.io.wavfile.write("reconstructed_pca3.wav",sample_rate,src3_pca)

We find out that the sources are not properly reconstructed and that there is indeed significant overlapp between the sources in the reconstructed signals. This is expected as PCA doesn't aim to decompose the signal to independent sources but rather it extracts the components that correspond to the direction of the maximal variance. Thus, in this case it is not the ideal solution for our problem and ICA clearly provides a better solution.